In [2]:
import numpy as np
import pandas as pd
import psycopg2

import matplotlib.pyplot as plt
import seaborn as sns

import matplotlib.style
matplotlib.style.use('ggplot')
%matplotlib inline


In [3]:
# create a database connection
sqluser = 'mimic'
dbname = 'mimic'
schema_name = 'mimiciii'

cur = None

In [190]:
if cur: 
    cur.close()
    con.close()

con = psycopg2.connect(dbname = dbname, user = sqluser)
cur = con.cursor()
cur.execute('SET search_path to ' + schema_name)

# Diagnoses

## Exclusion criteria: no cardiogenic shock, STEMI, PE or hemorrhagic shock

In [84]:
query = \
"""
SELECT * 
FROM d_icd_diagnoses
WHERE 
    long_title ~* '.*card.*' 
    OR long_title ~* '.*heart.*'
    OR long_title ~* '.*hemorrhag.*' 
    OR long_title ~* '.*bleed.*'
    OR long_title ~* '.*embolism.*'
"""
exc = pd.read_sql_query(query, con)
exc.shape

(778, 4)

In [77]:
exc.head()

,row_id,icd9_code,short_title,long_title
0,71,00804,Int inf e coli entrhmrg,Intestinal infection due to enterohemorrhagic ...
1,369,0860,Chagas disease of heart,Chagas' disease with heart involvement
2,416,09320,Syphil endocarditis NOS,"Syphilitic endocarditis of valve, unspecified"
3,417,09321,Syphilitic mitral valve,Syphilitic endocarditis of mitral valve
4,418,09322,Syphilitic aortic valve,Syphilitic endocarditis of aortic valve


Keep only those diagnoses that are relevant to the echos. To do that, first associate each echo to an icustay ID by tagging an echo to an ICU stay if the echo charttime occurred between (intime - 8 hours, outtime) of the ICU stay. Each echo should be linked to only 1 ICU stay. 

In [121]:
query = \
"""
WITH ei AS (
SELECT ed.row_id, ed.charttime, ie.icustay_id, ie.intime, ie.outtime, ie.hadm_id
FROM echodata ed
INNER JOIN icustays ie
    ON ie.hadm_id = ed.hadm_id
    WHERE (ed.charttime > (ie.intime - INTERVAL '8 hours')) AND (ed.charttime < ie.outtime)
)
SELECT * FROM ei
"""
ei = pd.read_sql_query(query, con)
ei.head()

,row_id,charttime,icustay_id,intime,outtime,hadm_id
0,77947,2118-09-02 13:39:00,241967,2118-09-02 11:18:44,2118-09-03 17:43:25,179179
1,77949,2185-03-24 14:00:00,212314,2185-01-27 16:15:51,2185-05-23 16:23:59,114624
2,77950,2185-03-23 15:00:00,212314,2185-01-27 16:15:51,2185-05-23 16:23:59,114624
3,77951,2185-03-18 15:00:00,212314,2185-01-27 16:15:51,2185-05-23 16:23:59,114624
4,78110,2130-02-10 13:00:00,279642,2130-02-10 15:27:29,2130-02-13 21:34:22,143061


In [122]:
ei['row_id'].value_counts().value_counts()

1    25512
Name: row_id, dtype: int64

In [125]:
len(ei['hadm_id'].unique())

19244

Cross-reference these hospital admission IDs to get all diagnoses that were made. 

In [329]:
query = \
"""
-- associate echos to icustays
WITH ei AS (
SELECT ed.row_id, ed.charttime, ie.icustay_id, ie.intime, ie.outtime, ie.hadm_id
FROM echodata ed
INNER JOIN icustays ie
    ON ie.hadm_id = ed.hadm_id
    WHERE (ed.charttime > (ie.intime - INTERVAL '8 hours')) AND (ed.charttime < ie.outtime)
), 
-- get all diagnoses associated with hospital admissions
-- that are associated with the above icustays
di AS (
    SELECT ei.icustay_id, di.*
    FROM diagnoses_icd di
    RIGHT JOIN ei -- keep only hadm_ids from ei
        ON di.hadm_id = ei.hadm_id
),
eid AS (
    -- filter diagnoses for keywords
    SELECT di.icustay_id, did.*
    -- may be helpful for the mds to list the keywords
        ,CASE WHEN did.long_title ~* '.*card.*' THEN 1 ELSE 0 END AS k_card
        ,CASE WHEN did.long_title ~* '.*heart.*' THEN 1 ELSE 0 END AS k_heart
        ,CASE WHEN did.long_title ~* '.*hemorrhag.*' THEN 1 ELSE 0 END AS k_hemorrhag
        ,CASE WHEN did.long_title ~* '.*bleed.*' THEN 1 ELSE 0 END AS k_bleed
        ,CASE WHEN did.long_title ~* '.*embolism.*' THEN 1 ELSE 0 END AS k_embolism
    FROM d_icd_diagnoses did 
    RIGHT JOIN di -- keep only icd9 codes from di
        ON did.icd9_code = di.icd9_code 
    WHERE 
           did.long_title ~* '.*card.*' 
        OR did.long_title ~* '.*heart.*'
        OR did.long_title ~* '.*hemorrhag.*' 
        OR did.long_title ~* '.*bleed.*'
        OR did.long_title ~* '.*embolism.*'
),
eidc AS (
    SELECT icd9_code
        ,COUNT(icd9_code) AS num
    FROM eid
    GROUP BY icd9_code
)
SELECT DISTINCT ON(eid.icd9_code) * from eid
INNER JOIN eidc
    ON eid.icd9_code = eidc.icd9_code
"""
did = pd.read_sql_query(query, con)
did.head()

,icustay_id,row_id,icd9_code,short_title,long_title,k_card,k_heart,k_hemorrhag,k_bleed,k_embolism,icd9_code,num
0,226474,638,03641,Meningococc pericarditis,Meningococcal pericarditis,1,0,0,0,0,03641,11
1,213981,559,0521,Varicella pneumonitis,Varicella (hemorrhagic) pneumonitis,0,0,1,0,0,0521,4
2,261941,369,0860,Chagas disease of heart,Chagas' disease with heart involvement,0,1,0,0,0,0860,1
3,287059,1412,11281,Candidal endocarditis,Candidal endocarditis,1,0,0,0,0,11281,6
4,217326,1198,1510,Mal neo stomach cardia,Malignant neoplasm of cardia,1,0,0,0,0,1510,21


In [330]:
did.shape

(380, 12)

In [9]:
query = \
"""
SELECT *
FROM d_diagnoses_xc;
    
"""
check = pd.read_sql_query(query, con)
check.head()

,icustay_id,row_id,icd9_code,short_title,long_title,k_card,k_heart,k_hemorrhag,k_bleed,k_embolism,num
0,226474,638,03641,Meningococc pericarditis,Meningococcal pericarditis,1,0,0,0,0,11
1,220601,559,0521,Varicella pneumonitis,Varicella (hemorrhagic) pneumonitis,0,0,1,0,0,4
2,261941,369,0860,Chagas disease of heart,Chagas' disease with heart involvement,0,1,0,0,0,1
3,210541,1412,11281,Candidal endocarditis,Candidal endocarditis,1,0,0,0,0,6
4,252388,1198,1510,Mal neo stomach cardia,Malignant neoplasm of cardia,1,0,0,0,0,21


In [10]:
check.sort_values('k_card', ascending = False).head()

,icustay_id,row_id,icd9_code,short_title,long_title,k_card,k_heart,k_hemorrhag,k_bleed,k_embolism,num
0,226474,638,03641,Meningococc pericarditis,Meningococcal pericarditis,1,0,0,0,0,11
120,261085,4495,42971,Acq cardiac septl defect,Acquired cardiac septal defect,1,0,0,0,0,74
98,211763,4471,42789,Cardiac dysrhythmias NEC,Other specified cardiac dysrhythmias,1,0,0,0,0,1472
99,237988,4472,4279,Cardiac dysrhythmia NOS,"Cardiac dysrhythmia, unspecified",1,0,0,0,0,8
115,267647,4488,4290,Myocarditis NOS,"Myocarditis, unspecified",1,0,0,0,0,16


## Check definition in comorbidities for congestive heart failure and renal failure

See [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3797549/). 
    
    

# Prescriptions

Filter for pressors and inotropes

In [233]:
query = \
"""
SELECT *
FROM d_prescriptions LIMIT 10;
    
"""
scripts = pd.read_sql_query(query, con)
scripts

,drug_type,drug,drug_name_poe,drug_name_generic,formulary_drug_cd,gsn,ndc,route
0,ADDITIVE,Albumin,None,None,ALBU25IVPB,006329,67467064301,IV
1,ADDITIVE,Albumin 25% (12.5g / 50mL),None,None,ALBU25,006329,67467064301,IV
2,ADDITIVE,Albumin 25% (12.5gm),None,None,ALBU25,006329,49669521303,IV
3,ADDITIVE,Albumin 25% (12.5gm),None,None,ALBU25,006329,52769025105,IV
4,ADDITIVE,Bupivacaine 0.05%,None,None,BUPI0.05EPD,None,0,ED
5,ADDITIVE,Bupivacaine 0.1%,None,None,BUPI0.1CADD,None,0,ED
6,ADDITIVE,Bupivacaine 0.1%,None,None,BUPI0.1EPD,None,0,ED
7,ADDITIVE,Bupivacaine 0.1%,None,None,BUPI0.1EPD,None,0,IV DRIP
8,ADDITIVE,Bupivacaine 0.1%,None,None,BUPI0.1EPD,None,0,IVPCA
9,ADDITIVE,Bupivacaine 0.1%,None,None,BUPI0.1EPD,None,0,LUMBAR PLEXUS


In [94]:
query = \
"""
SELECT COUNT(*)
FROM prescriptions;
    
"""
foo = pd.read_sql_query(query, con)
foo.head()

,count
0,4156450


To generate the dictionary d_prescriptions, select distinct on drug, drug_name_poe, drug_name_generic, formulary_drug_cd, gsn, ndc and route in the prescriptions table, and then search in drug, drug_name_poe and drug_name_generic for the keywords. 

In [105]:
query = \
"""
SELECT * FROM prescriptions
WHERE drug ~* '.*epinephrine.*'
"""

scripts = pd.read_sql_query(query, con)
scripts.shape

(16736, 19)

In [170]:
query = \
"""
-- search for keywords of interest
WITH prescriptions_k AS ( 
    SELECT *
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*epinephrine.*' THEN 1 ELSE 0 END AS k_epinephrine
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*levophed.*' THEN 1 ELSE 0 END AS k_levophed
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*neosynephrine.*' THEN 1 ELSE 0 END AS k_neosynephrine
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*isoprotenerol.*' THEN 1 ELSE 0 END as k_isoprotenerol
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*methylene blue.*' THEN 1 ELSE 0 END AS k_methyleneblue
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*dopamine.*' THEN 1 ELSE 0 END AS k_dopamine
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*phenylephrine.*' THEN 1 ELSE 0 END AS k_phenylephrine
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*vasopressin.*' THEN 1 ELSE 0 END AS k_vasopressin
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*adrenaline.*' THEN 1 ELSE 0 END AS k_adrenaline
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*isoprenaline.*' THEN 1 ELSE 0 END AS k_isoprenaline
        ,CASE WHEN concat(drug, drug_name_poe, drug_name_generic) ~* '.*terlipressin.*' THEN 1 ELSE 0 END AS k_terlipressin
    FROM prescriptions
)
-- apply filters
, prescriptions_kf AS ( 
    SELECT * FROM prescriptions_k 
    WHERE  (
           k_epinephrine = 1
        OR k_levophed = 1
        OR k_neosynephrine = 1
        OR k_isoprotenerol = 1
        OR k_methyleneblue = 1
        OR k_dopamine = 1
        OR k_phenylephrine = 1
        OR k_vasopressin = 1
        OR k_adrenaline = 1
        OR k_isoprenaline = 1
        OR k_terlipressin = 1
    ) AND route ~* '.*iv.*'
) 
-- get the annotations as a table
, prescriptions_annot AS (
    SELECT DISTINCT pk.drug, pk.drug_name_poe, pk.drug_name_generic, pk.route
        ,pk.k_epinephrine, pk.k_levophed, pk.k_neosynephrine, pk.k_isoprotenerol
        ,pk.k_methyleneblue, pk.k_dopamine, pk.k_phenylephrine, pk.k_vasopressin
        ,pk.k_adrenaline, pk.k_isoprenaline, pk.k_terlipressin
    FROM prescriptions_kf pk
)
-- select only entries where the hospital admission id matches those in echo_icustays
-- alternatively, select only entries where the icustay id matches those in echo_icustays
-- since we only care about the number of echos associated with each drug 
-- (here defined by (drug, drug_name_poe, drug_name_generic, route)
-- we can drop duplicates (for eg. in the case of the same drug being administered twice during an icustay)
, prescriptions_kfe AS (
    SELECT DISTINCT pk.icustay_id, pk.drug, pk.drug_name_poe, pk.drug_name_generic, pk.route 
    FROM prescriptions_kf pk
    INNER JOIN echo_icustay ei
        ON ei.icustay_id = pk.icustay_id
)
, prescriptions_count AS (
    SELECT drug, drug_name_poe, drug_name_generic, route, COUNT(*)
    FROM prescriptions_kfe
    GROUP BY drug, drug_name_poe, drug_name_generic, route
)
SELECT pa.*, pc.count FROM prescriptions_annot pa
RIGHT JOIN prescriptions_count pc
    ON (pa.drug IS NOT DISTINCT FROM pc.drug 
        AND pa.drug_name_poe IS NOT DISTINCT FROM pc.drug_name_poe
        AND pa.drug_name_generic IS NOT DISTINCT FROM pc.drug_name_generic
        AND pa.route IS NOT DISTINCT FROM pc.route)
        
"""

scripts = pd.read_sql_query(query, con)
scripts.sort_values('count', ascending = False)

,drug,drug_name_poe,drug_name_generic,route,k_epinephrine,k_levophed,k_neosynephrine,k_isoprotenerol,k_methyleneblue,k_dopamine,k_phenylephrine,k_vasopressin,k_adrenaline,k_isoprenaline,k_terlipressin,count
41,Phenylephrine,None,None,IV DRIP,0,0,0,0,0,0,1,0,0,0,0,3545
10,Norepinephrine,None,None,IV DRIP,1,0,0,0,0,0,0,0,0,0,0,2857
12,Phenylephrine HCl,None,None,IV DRIP,0,0,0,0,0,0,1,0,0,0,0,1611
44,Vasopressin,None,None,IV DRIP,0,0,0,0,0,0,0,1,0,0,0,1504
43,Epinephrine,None,None,IV DRIP,1,0,0,0,0,0,0,0,0,0,0,1018
47,PHENYLEPHrine,None,None,IV DRIP,0,0,0,0,0,0,1,0,0,0,0,971
8,NORepinephrine,None,None,IV DRIP,1,0,0,0,0,0,0,0,0,0,0,754
0,DOPamine,None,None,IV DRIP,0,0,0,0,0,1,0,0,0,0,0,669
27,DopAmine,None,None,IV DRIP,0,0,0,0,0,1,0,0,0,0,0,622
19,EPINEPHrine,None,None,IV DRIP,1,0,0,0,0,0,0,0,0,0,0,169


In [171]:
scripts.shape

(50, 16)

Equivalently, see `d_prescriptions_vaso`.

In [185]:
query = \
"""
SELECT * FROM d_prescriptions_vaso
"""

scripts = pd.read_sql_query(query, con)
scripts.head()

,drug,drug_name_poe,drug_name_generic,route,k_epinephrine,k_levophed,k_neosynephrine,k_methyleneblue,k_dopamine,k_phenylephrine,k_vasopressin,k_adrenaline,k_isoprenaline,k_terlipressin,count
0,Dopamine HCl,None,None,IV,0,0,0,0,1,0,0,0,0,0,3
1,DopAmine,None,None,IV DRIP,0,0,0,0,1,0,0,0,0,0,471
2,Phenylephrine,Phenylephrine,Phenylephrine,IV DRIP,0,0,0,0,0,1,0,0,0,0,10
3,PHENYLEPHrine,PHENYLEPHrine,PHENYLEPHrine,IV,0,0,0,0,0,1,0,0,0,0,1
4,Methylene Blue,None,None,IV,0,0,0,1,0,0,0,0,0,0,8


In [186]:
keywords = [k for k in scripts.columns if k[0] == 'k']
scripts[keywords].sum()

k_epinephrine      18
k_levophed          0
k_neosynephrine     0
k_methyleneblue     5
k_dopamine          8
k_phenylephrine    12
k_vasopressin       4
k_adrenaline        0
k_isoprenaline      0
k_terlipressin      0
dtype: int64

We may also just be interested in the total number of echos associated to any one of these vaso-related drugs. 

In [184]:
query = \
"""
WITH echo_ps AS (
    SELECT DISTINCT ep.icustay_id
    FROM prescriptions ep
    INNER JOIN echo_icustay ei
        ON ei.icustay_id = ep.icustay_id
    WHERE (concat(ep.drug, ep.drug_name_poe, ep.drug_name_generic) ~* '.*epinephrine.*'
        OR concat(ep.drug, ep.drug_name_poe, ep.drug_name_generic) ~* '.*levophed.*'
        OR concat(ep.drug, ep.drug_name_poe, ep.drug_name_generic) ~* '.*neosynephrine.*'
        OR concat(ep.drug, ep.drug_name_poe, ep.drug_name_generic) ~* '.*methylene blue.*'
        OR concat(ep.drug, ep.drug_name_poe, ep.drug_name_generic) ~* '.*dopamine.*'
        OR concat(ep.drug, ep.drug_name_poe, ep.drug_name_generic) ~* '.*phenylephrine.*'
        OR concat(ep.drug, ep.drug_name_poe, ep.drug_name_generic) ~* '.*vasopressin.*'
        OR concat(ep.drug, ep.drug_name_poe, ep.drug_name_generic) ~* '.*isoprenaline.*'
        OR concat(ep.drug, ep.drug_name_poe, ep.drug_name_generic) ~* '.*terlipressin.*'
    ) AND route ~* '.*iv.*'
)
SELECT COUNT(*) FROM echo_ps;
"""

count = pd.read_sql_query(query, con)
count

,count
0,7035


# Labs

In [290]:
query = \
"""
SELECT * 
FROM d_labitems
WHERE row_id IN (
     '21' -- pH
    ,'63' -- Albumin
    ,'83' -- Bicarbonate
    ,'90' -- C-Reactive Protein
    ,'488' -- Sedimentation Rate
    ,'113' -- Creatinine
    ,'109' -- CK-MB Index
    ,'111' -- Creatine Kinase (CK)
    ,'112' -- Creatine kinase (MB Isoenzyme)
    ,'121' -- Estimted GFR (MDRD Equation)
    ,'202' -- Troponin I
    ,'203' -- Troponin T
    ,'500' -- WBC Count
    ,'501' -- White Blood Cells
    ,'11' -- Hematocrit, Calculated
    ,'12' -- Hemoglobin
    ,'14' -- Lactate
    ,'164' -- NTProBNP
)
"""
labs = pd.read_sql_query(query, con)
labs

,row_id,itemid,label,fluid,category,loinc_code
0,11,50810,"Hematocrit, Calculated",Blood,Blood Gas,20570-8
1,12,50811,Hemoglobin,Blood,Blood Gas,718-7
2,14,50813,Lactate,Blood,Blood Gas,32693-4
3,21,50820,pH,Blood,Blood Gas,11558-4
4,63,50862,Albumin,Blood,Chemistry,1751-7
5,83,50882,Bicarbonate,Blood,Chemistry,1963-8
6,90,50889,C-Reactive Protein,Blood,Chemistry,1988-5
7,109,50908,CK-MB Index,Blood,Chemistry,20569-0
8,111,50910,Creatine Kinase (CK),Blood,Chemistry,2157-6
9,112,50911,"Creatine Kinase, MB Isoenzyme",Blood,Chemistry,6773-6


Also check items_d, 

In [304]:
query = \
"""
SELECT * 
FROM d_items
WHERE category ~* '.*labs.*'
"""
items = pd.read_sql_query(query, con)
items.shape

(148, 10)

In [314]:
query = \
"""
SELECT DISTINCT category
FROM d_items
"""
cats = pd.read_sql_query(query, con)
cats.iloc[:,0].values

array([None, 'Drug Level', 'Meds', 'SPECIMEN', 'Nutrition - Enteral',
       'Nutrition - Supplements', "VBG'S", 'Fluids - Other (Not In Use)',
       'Skin - Assessment', 'Urine', 'Respiratory', 'OB-GYN', 'Labs',
       'OT Notes', 'Research Enrollment Note', 'Medications', 'Heme/Coag',
       '5-Imaging', 'NICOM', 'ABG', 'CSF', 'Blood Products/Colloids',
       'Pastoral Care Note', 'Other ABGs', 'Cardiovascular (Pulses)',
       'Antibiotics', 'NG Feeding', 'Access Lines - Peripheral',
       'Thoracentesis', 'Routine Vital Signs', 'Case Management',
       'Blood Gases', '6-Cultures', 'Generic Proc Note', 'Coags', 'Output',
       'CVL Insertion', 'Quick Admit', 'Scores - APACHE IV (2)',
       'Access Lines - Invasive', 'Cardiovascular (Pacer Data)',
       'Toxicology', "ABG's", "ABG'S", 'GI/GU', 'Tandem Heart', 'Enzymes',
       'Neurological', 'Chemistry', 'ApacheIV Parameters', 'Hemodynamics',
       'PA Line Insertion', '1-Intubation/Extubation',
       'ApacheII Parameters',

# Counts with time_filter on in echo_icustay

Note that the counts above were calculated with the time filter (-8 to 48 hours) on. 

In [188]:
query = \
"""
SELECT *
FROM echo_icustay
"""
echos = pd.read_sql_query(query, con)
echos.head()

,row_id,charttime,icustay_id,intime,outtime,hadm_id,time_to_echo,time_filter
0,77947,2118-09-02 13:39:00,241967,2118-09-02 11:18:44,2118-09-03 17:43:25,179179,0 days 02:20:16,True
1,77949,2185-03-24 14:00:00,212314,2185-01-27 16:15:51,2185-05-23 16:23:59,114624,55 days 21:44:09,False
2,77950,2185-03-23 15:00:00,212314,2185-01-27 16:15:51,2185-05-23 16:23:59,114624,54 days 22:44:09,False
3,77951,2185-03-18 15:00:00,212314,2185-01-27 16:15:51,2185-05-23 16:23:59,114624,49 days 22:44:09,False
4,78110,2130-02-10 13:00:00,279642,2130-02-10 15:27:29,2130-02-13 21:34:22,143061,-1 days +21:32:31,True


In [189]:
echos['time_filter'].value_counts()

True     18811
False     6701
Name: time_filter, dtype: int64

Of which 7035 are on vaso-related drugs. 